# Chapter-9 Standard Libraries
This notebook contains the sample source code explained in the book *Hands-On Julia Programming, Sambit Kumar Dash, 2021, bpb Publications. All Rights Reserved*.

In [1]:
using Pkg
pkg"activate ."
pkg"instantiate"

  Activating environment at `~/work/books/HOPJ/Chapter-09/Project.toml`


## 9.1 Introduction

There are various functions and methods that are part of Julia already. A developer needs to use them directly rather than having to code for them. This chapter is a bird's eyeview of such methods. 

### Modules

Namespaces where a method or a type reside. Using a method or type outside the module may require explicit qualification. 

In [2]:
module m1
    function f()
        println("Defined as m1.f")
    end
    function g()
        println("Defined as m1.g")
    end
    function h()
        println("Defined as m1.h")
    end
    export f
end

module m2
    function f()
        println("Defined as m2.f")
    end
    function g()
        println("Defined as m2.g")
    end
    export g
end

Main.m2

In [3]:
names(m1)

2-element Vector{Symbol}:
 :f
 :m1

In [4]:
names(m2)

2-element Vector{Symbol}:
 :g
 :m2

In [5]:
f()

LoadError: UndefVarError: f not defined

In [6]:
m1.f()

Defined as m1.f


In [7]:
m2.f()

Defined as m2.f


In [8]:
using .m1

In [9]:
f()

Defined as m1.f


In [10]:
g()

LoadError: UndefVarError: g not defined

In [11]:
using Main.m2

In [12]:
g()

Defined as m2.g


In [13]:
using .m1: g

In [14]:
g()

Defined as m2.g


In [15]:
using .m1: h

In [16]:
h()

Defined as m1.h


## 9.2 Standard Modules

`Core`, `Base` and `Main` the bare minimum modules of a Julia shell. `m1` and `m2` here are submodules of `Main`.

In [17]:
varinfo()

| name |      size | summary |
|:---- | ---------:|:------- |
| Base |           | Module  |
| Core |           | Module  |
| Main |           | Module  |
| m1   | 6.489 KiB | Module  |
| m2   | 5.608 KiB | Module  |


In [18]:
x = 1

1

In [19]:
varinfo()

| name |      size | summary |
|:---- | ---------:|:------- |
| Base |           | Module  |
| Core |           | Module  |
| Main |           | Module  |
| m1   | 6.489 KiB | Module  |
| m2   | 5.608 KiB | Module  |
| x    |   8 bytes | Int64   |


## 9.3 System

The submodule that interacts with the operating system. System command execution being one of them. 

In [20]:
run(`cmd /c dir`)

LoadError: IOError: could not spawn `cmd /c dir`: no such file or directory (ENOENT)

In [21]:
Sys.isunix(), Sys.iswindows()

(true, false)

## 9.4 Filesystem

File organization and manipulation (not read and write). 

In [22]:
old_dir = pwd()

"/home/sambit/work/books/HOPJ/Chapter-09"

In [23]:
cd("..")

In [24]:
pwd()

"/home/sambit/work/books/HOPJ"

In [25]:
cd(old_dir)

In [26]:
pwd()

"/home/sambit/work/books/HOPJ/Chapter-09"

In [27]:
cd(".."); pwd()

"/home/sambit/work/books/HOPJ"

In [28]:
readdir()  #Same as readdir(pwd())

20-element Vector{String}:
 ".git"
 ".github"
 ".gitignore"
 "Chapter-02"
 "Chapter-03"
 "Chapter-04"
 "Chapter-05"
 "Chapter-06"
 "Chapter-07"
 "Chapter-08"
 "Chapter-09"
 "Chapter-10"
 "Chapter-11"
 "Chapter-12"
 "Chapter-13"
 "Chapter-14"
 "Chapter-15"
 "Chapter-16"
 "Chapter-17"
 "manifests"

In [29]:
for (root, dirs, files) in walkdir("Chapter-09")
    println("Directories in $root")
    for dir in dirs
        println(joinpath(root, dir)) # path to directories
    end
    println("Files in $root")
    for file in files
        println(joinpath(root, file)) # path to files
    end
end

Directories in Chapter-09
Chapter-09/.ipynb_checkpoints
Files in Chapter-09
Chapter-09/Chapter-9 Standard Libraries.ipynb
Chapter-09/Manifest.toml
Chapter-09/Project.toml
Directories in Chapter-09/.ipynb_checkpoints
Files in Chapter-09/.ipynb_checkpoints
Chapter-09/.ipynb_checkpoints/Chapter-9 Standard Libraries-checkpoint.ipynb


In [30]:
cd(old_dir); pwd()

"/home/sambit/work/books/HOPJ/Chapter-09"

In [31]:
fs = stat("Chapter-9 Standard Libraries.ipynb")

StatStruct(mode=0o100664, size=64819)

## 9.5 Parallel Programming

Execution of programs in tandem in various execution environments, yet be able to obtain meaningful and synchronized results. Performance and resource utilization are two major outcomes of this kind of programming. 

### Asynchronous Programming

Breaking the code into small chunks of executable pieces that can be executed in parallel. Depending on the kind of processing permitted by the hardware and OS, these may run in one execution thread or multiple.

#### Tasks

In [32]:
t = Task() do 
    sleep(10)
    println("done")
end

Task (runnable) @0x00007f916a6e5270

In [33]:
schedule(t)

Task (runnable) @0x00007f916a6e5270

In [34]:
t = Task() do 
    sleep(10)
    println("done")
end

Task (runnable) @0x00007f913ff52e00

In [35]:
schedule(t); wait(t)

done
done


In [36]:
t = @task begin
    sleep(5)
    println("done")
end

Task (runnable) @0x00007f91682b30a0

In [37]:
@sync  begin
    sleep(5)
    println("done")
end

done


#### Channels

In [38]:
c = Channel(8)

Channel{Any}(8) (empty)

In [39]:
c = Channel{Int}(4)

Channel{Int64}(4) (empty)

In [40]:
@async begin
    for i = 1:100
        println("Adding $i to channel")
        put!(c, i)
    end
end

@async begin
    for i = 1:100
        v = take!(c)
        println("Removing value: $v")
    end
end

Adding 1 to channel


Task (runnable) @0x00007f913f65d120

Adding 2 to channel
Adding 3 to channel
Adding 4 to channel
Adding 5 to channel
Adding 6 to channel
Removing value: 1
Removing value: 2
Removing value: 3
Removing value: 4
Removing value: 5
Removing value: 6
Adding 7 to channel
Adding 8 to channel
Adding 9 to channel
Adding 10 to channel
Adding 11 to channel
Adding 12 to channel
Removing value: 7
Removing value: 8
Removing value: 9
Removing value: 10
Removing value: 11
Removing value: 12
Adding 13 to channel
Adding 14 to channel
Adding 15 to channel
Adding 16 to channel
Adding 17 to channel
Adding 18 to channel
Removing value: 13
Removing value: 14
Removing value: 15
Removing value: 16
Removing value: 17
Removing value: 18
Adding 19 to channel
Adding 20 to channel
Adding 21 to channel
Adding 22 to channel
Adding 23 to channel
Adding 24 to channel
Removing value: 19
Removing value: 20
Removing value: 21
Removing value: 22
Removing value: 23
Removing value: 24
Adding 25 to channel
Adding 26 to channel
Adding 27 to channel
Adding 28 to ch

In [41]:
close(c)

### Multithreading

Julia can be made to run on a multiple threads mode. In such a conditions the tasks can be parallelized on these threads. Synchronization objects can be used across the executing threads. 

In [42]:
Threads.nthreads()

1

In [43]:
Threads.@spawn for i=1:100
    sleep(1)
    println("Step: $i")
end

Task (runnable) @0x00007f913ff13490

Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
Step: 85
Step: 86
Step: 87
Step: 88
Step: 89
Step: 90
Step: 91
Step: 92
Step: 93
Step: 94
Step: 95
Step: 96
Step: 97
Step: 98
Step: 99
Step: 100


### Distributed Computing

This is cluster computing infrastrcuture, where a cluster can be configured across Julia processes in the same machine or in another machine. These independent processes can execute independently yet be able to share data, pass messages and communicate over a shared channel. 

## 9.6 IO and Network

Julia accesses input and output devices as streams. The network devices are also interfaced like sockets. IO is an important stream based interface to understand. 

### Default Stream

`stdin`, `stdout` and `stderr` are the standard streams for input, output and error in most processes. `stdin` integration with Jupyter is limited. All those samples may best be reviewed in a REPL console. 

In [44]:
write(stdout, "Hello World")

Hello World

11

In [45]:
write(stdout, "\u2200 x \u2203 y")

∀ x ∃ y

11

### Text I/O

### Binary I/O

### File I/O

Files are also accessed as `IO` streams for both read and write operations. However, the `IO` streams are obtained by `open` call. Any open stream must be closed with a `close` function call. 

In [46]:
write("hello.txt", "Hello World")

11

In [47]:
fd = open("hello.txt", "r");
data = read(fd);

In [48]:
write(fd, "This is second line")

LoadError: ArgumentError: write failed, IOStream is not writeable

In [49]:
close(fd)
write(stdout, data)

Hello World

11

In [50]:
open("hello.txt", "r") do fd
    data = read(fd)
    write(stdout, data)
end

Hello World

11

In [51]:
write("hello.txt", "Hello World");
data = read("hello.txt");
write(stdout, data);

Hello World

Files are opened for `read` or `write`. A file opened for reading cannot be written into.  

In [52]:
open("hello.txt", "r") do fd
    while !eof(fd)
        b = read(fd, 1)
        write(stdout, b)
    end
end
rm("hello.txt")

Hello World

### Delimited Files

Generalized command separated value (CSV) kind of files. Used extensively in spreadsheets. Also interfaced in data science applications as an easier way to share data. 

In [53]:
using DelimitedFiles
x = collect(1:100);
y = Float64.(collect(101:200));
open("values.csv", "w") do f
    println(f, "X,Y")
    writedlm(f, [x y], ',')
end

In [54]:
data, header = readdlm("values.csv", ',', header=true);

In [55]:
size(data, 1)

100

In [56]:
header

1×2 Matrix{AbstractString}:
 "X"  "Y"

In [57]:
rm("values.csv")

### Network 

Just like files are opened with open to obtain the `IO` stream to interface, sockets are equivalent network communication end points to obtain connection streams. However, if you could use download method to provide a URL and download files. 

In [58]:
download("https://github.com/JuliaLang/julia/releases/download/v1.5.3/julia-1.5.3.tar.gz", "julia-1.5.3.tar.gz")

"julia-1.5.3.tar.gz"

In [59]:
using Sockets
@async begin
    server = listen(3000)
    while true
        s = accept(server)
        @async while isopen(s)
            msg = readline(s, keep=true)
            write(s, "Server Response: $msg")
        end  
    end
end

Task (runnable) @0x00007f913b0bcfd0

In [60]:
c = connect(3000)
@async while isopen(c)
    println(stdout, readline(c, keep=true))
end

Task (runnable) @0x00007f913fe63880

In [61]:
for i = 1:5
    println(c, "Client Msg id: $i")
end

Server Response: Client Msg id: 1

Server Response: Client Msg id: 2

Server Response: Client Msg id: 3

Server Response: Client Msg id: 4

Server Response: Client Msg id: 5



In [62]:
close(c)

### Memory I/O

Strings cannot be manipulated. It may help to create a byte array for `IO` in the memory and use that for text manipulations. In the end the byte array can be converted to a String. 

In [63]:
io = IOBuffer()
println(io, "This is Line 1")
println(io, "This is Line 2")
str = String(take!(io))

"This is Line 1\nThis is Line 2\n"

In [64]:
A = fill(10, (5, 20))
A[2, 3] = 20
open("mmap.bin", "w+") do fd
    write(fd, size(A, 1))
    write(fd, size(A, 2))
    write(fd, A)
end

800

Memory mapping is reverse problem. A file can be mapped to memory and accessed selectively page by page. This way the whole file need not be loaded in the memory. 

In [65]:
using Mmap
fd = open("mmap.bin", "r")
nr = read(fd, Int)
nc = read(fd, Int)
A2 = Mmap.mmap(fd, Matrix{Int}, (nr, nc))
println("A2[1, 1]: ", A2[1, 1], " A2[2, 3]: ", A2[2, 3])
close(fd)

A2[1, 1]: 10 A2[2, 3]: 20


In [66]:
A2 = nothing
GC.gc()
rm("mmap.bin")

## 9.7 Constants

There are many system and Julia related constants. Mathematical constants are also defined in the standard library. 

In [67]:
VERSION

v"1.6.0"

In [68]:
C_NULL

Ptr{Nothing} @0x0000000000000000

In [69]:
Sys.BINDIR

"/home/sambit/work/deps/julia/julia-1.6.0/bin"

In [70]:
Sys.CPU_THREADS

4

In [71]:
Sys.WORD_SIZE

64

In [72]:
Sys.KERNEL

:Linux

In [73]:
Sys.MACHINE

"x86_64-pc-linux-gnu"

In [74]:
Sys.ARCH

:x86_64

In [75]:
Sys.ENV

Base.EnvDict with 71 entries:
  "SHELL"                    => "/bin/bash"
  "SESSION_MANAGER"          => "local/ubutubu:@/tmp/.ICE-unix/9757,unix/ubutub…
  "QT_ACCESSIBILITY"         => "1"
  "COLORTERM"                => "truecolor"
  "XDG_CONFIG_DIRS"          => "/etc/xdg/xdg-ubuntu:/etc/xdg"
  "XDG_MENU_PREFIX"          => "gnome-"
  "GNOME_DESKTOP_SESSION_ID" => "this-is-deprecated"
  "CONDA_EXE"                => "/home/sambit/anaconda3/bin/conda"
  "_CE_M"                    => ""
  "MANDATORY_PATH"           => "/usr/share/gconf/ubuntu.mandatory.path"
  "JAVA_HOME"                => "/home/sambit/java"
  "GNOME_SHELL_SESSION_MODE" => "ubuntu"
  "JRE_HOME"                 => "/home/sambit/java"
  "SSH_AUTH_SOCK"            => "/run/user/1000/keyring/ssh"
  "XMODIFIERS"               => "@im=ibus"
  "DESKTOP_SESSION"          => "ubuntu"
  "SSH_AGENT_PID"            => "9715"
  "GTK_MODULES"              => "gail:atk-bridge"
  "PWD"                      => "/home/sambit/work/boo

In [76]:
MathConstants.pi

π = 3.1415926535897...

In [77]:
MathConstants.e

ℯ = 2.7182818284590...

## 9.8 Notable Modules

Miscellaneous modules of Julia library. 

### Dates

User renderable date and time representations. In the system the date is just a continuously increasing counter from a datum. These functions make meaningful period associations and provide the correct date and time. 

In [78]:
using Dates

In [79]:
now()

2021-03-31T02:14:32.320

In [80]:
DateTime(2020)

2020-01-01T00:00:00

In [81]:
t = DateTime(2020, 10, 31, 01, 02, 03)

2020-10-31T01:02:03

In [82]:
Date(2020)

2020-01-01

In [83]:
d = Date(t)

2020-10-31

In [84]:
DateTime(d)

2020-10-31T00:00:00

In [85]:
t > d

true

In [86]:
DateTime(2020) < now()

true

In [87]:
DateTime(2020) == Date(2020)

true

In [88]:
p = Month(3) + Day(10) + Hour(15)

3 months, 10 days, 15 hours

In [89]:
typeof(p)

Dates.CompoundPeriod

In [90]:
now() + p

2021-07-10T17:14:53.729

In [91]:
Date(now())+ Day(300)

2022-01-25

In [92]:
now() - DateTime(2020)

39320096409 milliseconds

In [93]:
now() - Date(2020)

LoadError: MethodError: no method matching -(::DateTime, ::Date)
[0mClosest candidates are:
[0m  -([91m::StridedArray{var"#s832", N} where {var"#s832"<:Union{Dates.CompoundPeriod, Period}, N}[39m, ::TimeType) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Dates/src/deprecated.jl:26
[0m  -(::TimeType, [91m::Dates.CompoundPeriod[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Dates/src/periods.jl:384
[0m  -(::T, [91m::T[39m) where T<:TimeType at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Dates/src/arithmetic.jl:9
[0m  ...

In [94]:
Date(now()) - Date(2020)

455 days

In [95]:
Year(now())

2021 years

In [96]:
Month(now())

3 months

In [97]:
Day(now())

31 days

In [98]:
Week(now())

13 weeks

In [99]:
year(now())

2021

In [100]:
tonext(today()) do d
    dayofweek(d) == Thursday &&
    dayofweekofmonth(d) == 4 &&
    month(d) == November
end

2021-11-25

### Logging

Long running processes or servers running in Julia need to provide consistent and standard log reporting. This module helps create such reports. 

In [101]:
@debug "This is a debug message $(sum(rand(100)))"

In [102]:
@info "This message is just informational"

┌ Info: This message is just informational
└ @ Main In[102]:1


In [103]:
@warn "This is a warning message"

┌ Warning: This is a warning message
└ @ Main In[103]:1


In [104]:
@error "This is an error message"

┌ Error: This is an error message
└ @ Main In[104]:1


In [105]:
A = ones(3, 4)
@info "A is all ones" A

┌ Info: A is all ones
│   A = [1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 1.0]
└ @ Main In[105]:2


The `Logging` module provides interfaces to implement complete loggers. It also provides the ability to override the global logger with a different local one. 

In [106]:
using Logging

In [107]:
open("file.log", "w") do f
    with_logger(SimpleLogger(f)) do
        @info "This is my simple info log"
        @error "This is an error written to file"
        @warn "This is a warning message"
        @info "Current logger is same as global logger" current_logger() == global_logger()
    end
end
write(stdout, read("file.log"));

┌ Info: This is my simple info log
└ @ Main In[107]:3
┌ Error: This is an error written to file
└ @ Main In[107]:4
┌ Warning: This is a warning message
└ @ Main In[107]:5
┌ Info: Current logger is same as global logger
│   current_logger() == global_logger() = false
└ @ Main In[107]:6


In [108]:
rm("file.log")

In [109]:
@info "Current logger is same as global logger" current_logger() == global_logger()

┌ Info: Current logger is same as global logger
│   current_logger() == global_logger() = true
└ @ Main In[109]:1


### Statistics

This module provides simple functions for central tendencies like mean, median and standard deviations etc. Functions for correlations and covariances are also provided. However, the treatment of statistics in thi module are highly rudimentary. 

In [110]:
using Statistics

In [111]:
mean(1:10)

5.5

In [112]:
median(1:10)

5.5

In [113]:
std(1:10)

3.0276503540974917

In [114]:
var(1:10)

9.166666666666666

In [115]:
cor(1:10, rand(10))

-0.005045481677433136

In [116]:
cor(1:10, 11:20)

1.0

In [117]:
cov(1:10, rand(10))

0.3460879740119759

### Random Numbers

Provides methods for random number generation and other permutation and combination functions. 

In [118]:
using Random

In [119]:
rand(Int, 2)

2-element Vector{Int64}:
 -3683361872024567216
  6382831596640906158

In [120]:
rand(2, 3)

2×3 Matrix{Float64}:
 0.26227  0.0469298  0.0352454
 0.35536  0.271616   0.545435

In [121]:
rand!(zeros(2, 3))

2×3 Matrix{Float64}:
 0.52666   0.393512  0.674182
 0.156176  0.117102  0.700993

In [122]:
rand(1:4, (2, 3))

2×3 Matrix{Int64}:
 1  4  3
 3  3  1

## 9.9 Conclusion

## Exercises